<a href="https://colab.research.google.com/github/ng13/AI_wrkshp/blob/main/35_Chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![NVIDIA](images/nvidia.png)

# Chatbots

In this notebook, you will learn how to store conversation history and thereby enable chatbot functionality in your LLM-based chains.

---

## Objectives

By the time you complete this notebook you will:

- Understand the core principles and techniques required to create chatbot applications capable of retaining conversation history.
- Create easy-to-use chatbots, capable of assuming a variety of different personas.
- Interact with a simple chatbot application interface, well-suited for chatbot application prototyping.

---

## Imports

In [ ]:
!pip install langchain-core langchain-nvidia-ai-endpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## Create a Model Instance

In [ ]:
llm = ChatNVIDIA(
  model="meta/llama-3.1-8b-instruct",
  api_key="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",
  temperature=0.2,
  top_p=0.7,
  max_completion_tokens=1024,
)

---

## Placeholder Messages

Before we begin work enabling conversation history and chatbot functionality, we need to introduce a new kind of message that we have not yet covered, **placeholder** messages.

Put simply, and as the name suggest, a placeholder message is used in a prompt template to hold the place of a list of other messages.

In [ ]:
template_with_placeholder = ChatPromptTemplate.from_messages([
    ('placeholder', '{messages}'),
    ('human', '{prompt}')
])

In [ ]:
messages = [
    ('human', 'The sun came up today.'),
    ('ai', 'That is wonderful!'),
    ('human', 'The sun went down today.'),
    ('ai', 'That is also wonderful!.')
]

In [ ]:
prompt = 'What happened today?'

When invoking (or streaming or batching) prompt templates or chains that contain placeholder messages, we provide a value as the template indicates, only in the case of a placeholder message, we provide a list of other messages, rather than a string.

Here we invoke `template_with_placeholder`, passing in the `messages` list to fulfil the template's `messages` parameter, and the `prompt` string to fulfil its `prompt` parameter.

In [ ]:
template_with_placeholder.invoke({'messages': messages, 'prompt': prompt})

ChatPromptValue(messages=[HumanMessage(content='The sun came up today.'), AIMessage(content='That is wonderful!'), HumanMessage(content='The sun went down today.'), AIMessage(content='That is also wonderful!.'), HumanMessage(content='What happened today?')])

As you can see, LangChain expanded the `placeholder` message, for which we provided a list, into a list of individual messages that we provided when invoking the prompt template.

It should come as no surprise that we can use this prompt template in chains just as we would any other.

In [ ]:
chain = template_with_placeholder | llm | StrOutputParser()

In [ ]:
chain.invoke({'messages': messages, 'prompt': prompt})

"It sounds like the sun rose and set, which is a normal part of the day-night cycle. That's a pretty ordinary day! Did anything else notable happen today?"

---

## Rudimentary Conversation History

We can easily construct a rudimentary conversation history mechanism using a message placeholder. First we'll create a prompt template utilizing a placeholder, and use it in a simple chain.

In [ ]:
chat_conversation_template = ChatPromptTemplate.from_messages([
    ('placeholder', '{chat_conversation}')
])

In [ ]:
chat_chain = chat_conversation_template | llm | StrOutputParser()

Next we'll create a list to store our conversation, which we will add to over time.

In [ ]:
chat_conversation = []

We will begin by appending our first prompt, as a `user` message to the `chat_conversation` list.

In [ ]:
chat_conversation.append(('user', 'Hello, my name is Michael.'))

Just to test it out, we can now invoke our `chat_chain` with the current `chat_conversation` list.

In [ ]:
chat_chain.invoke({'chat_conversation': chat_conversation})

"Hello Michael! It's nice to meet you. Is there something I can help you with or would you like to chat?"

It looks like the LLM is able to respond just fine. Since we are wanting to keep track of the conversation history, however, let's invoke the chain again, but this time append the response to the `chat_conversation` list as an `ai` message.

In [ ]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(('ai', response))

Looking at `chat_conversation` we see it now contains a list of the messages thus far.

In [ ]:
chat_conversation

[('user', 'Hello, my name is Michael.'),
 ('ai',
  "Hello Michael! It's nice to meet you. Is there something I can help you with or would you like to chat?")]

Let's repeat this same process with a new message, and let's pass in a prompt that relies on previous conversation history to answer correctly.

In [ ]:
chat_conversation.append(('user', 'Do you remember what my name is?'))

In [ ]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(('ai', response))
chat_conversation

[('user', 'Hello, my name is Michael.'),
 ('ai',
  "Hello Michael! It's nice to meet you. Is there something I can help you with or would you like to chat?"),
 ('user', 'Do you remember what my name is?'),
 ('ai',
  "Your name is Michael. I'll remember it for our conversation. How's your day going so far?")]

As you can see, by appending user prompt and AI responses to `chat_conversation` as `user` and `ai` messages respectively, and then invoking our placeholder-containing `chat_chain` with the entire up-to-date conversation, we now have the ability to converse with the LLM in a way where it retains details from earlier in the conversation.

At its most basic level, all chatbot functionality that is capable of retaining conversation history utilizes this method of passing in the conversation prior to new user messages.

---

## Chatbot Class

We can encapsulate the functionality we acheived above into a class that will make interacting with our conversation history-enabled LLM much simpler. Please read following `Chatbot` class definition, including its comments, carefully.

In [ ]:
class Chatbot:
    def __init__(self, llm):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))

        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

Let's instantiate a chatbot instance.

In [ ]:
chatbot = Chatbot(llm)

We can now utilize its `chat` method.

In [ ]:
print(chatbot.chat('Hi, my name is Michael.'))

Hi Michael! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [ ]:
print(chatbot.chat('I just want to be reminded of my name please.'))

Your name is Michael.


In [ ]:
print(chatbot.chat("Tell me something interesting I probably don't know about pi."))

A math enthusiast, eh? Here's something interesting:

Did you know that pi (π) is an irrational number, which means it can't be expressed as a finite decimal or fraction. However, what's even more fascinating is that pi is also a transcendental number, which means it's not the root of any polynomial equation with rational coefficients. In other words, there's no simple formula that can express pi as a finite combination of integers and roots.

But here's the really cool part: pi is connected to the Fibonacci sequence! The digits of pi are not randomly distributed, and in fact, the frequency of certain digit combinations in pi is similar to the frequency of those combinations in the Fibonacci sequence. This is known as the "Fibonacci hypothesis" or "Benford's law" for pi. It's still an open question in mathematics whether this is a coincidence or a deeper mathematical connection.

Mind blown, Michael?


In [ ]:
print(chatbot.chat("That's really cool! Give me another.")) # Note we are not being specific about what "another" refers to...the LLM needs to have previous messages to understand our intent.

Here's another one:

Did you know that pi is connected to the geometry of the universe? The ratio of a circle's circumference to its diameter is pi, and this ratio appears in many natural phenomena, such as:

* The shape of galaxies and galaxy clusters
* The arrangement of leaves on stems
* The branching of trees
* The flow of water in rivers and streams
* The structure of DNA

In fact, pi is a fundamental constant that appears in many areas of mathematics, physics, and engineering, including:

* The laws of gravity and motion (Einstein's general relativity)
* The behavior of waves and oscillations (acoustics and electromagnetism)
* The design of electronic circuits and computer chips
* The study of population growth and epidemiology

Pi is like a thread that weaves through many different areas of mathematics and science, connecting seemingly unrelated phenomena and patterns.

Pretty mind-blowing, right, Michael?


---

## More Advanced Chatbots

The topic of managing conversation history, and therefore of creating chatbots is actually quite large and there are many more advanced techniques that are outside the scope of this workshop. We do, however, want to provide you with some additional references for further study, should you wish to pursue the topic further.

- [Session-based conversation history, and history trimming](https://python.langchain.com/docs/how_to/chatbots_memory/): LangChain ships with tooling for wrapping chains in a way that provides history management, and specifically, managing multiple conversation sessions. This resource introduces the use of LangChain tooling for managing session-based conversation history, and also covers some techniques for managing the length of conversation history via message trimming and summarization, an important topic as chat conversations may grow large, or even too large to continue passing into an LLM.
- [Conversational RAG](https://python.langchain.com/docs/tutorials/qa_chat_history/): Retrieval Augmented Generation, or RAG (see [this DLI self-paced course](https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+S-FX-15+V1) for more) is a technique whereby LLMs can be provided real-time context from external data sources in support generating thier response. This resource discusses RAG in the context of chatbots capable of retaining conversation history.

---

## Exercise: Enable Role-based Chatbots

For this exercise you'll enable your chatbot instances to assume a specific role by leveraging a system message.

Below is the class definition for `ChatbotWithRole` which currently is identical to the `Chatbot` class definition above except that we've defined a `system_message` argument (defaulting to an empty string) that be used when instantiating `ChatbotWithRole` instances.

Edit the class definition as needed to that you can supply a system message that will create a specific role for your chatbot. Upon completion you should be able to use system messages like the following to create an overarching role for your chatbot to assume.

Feel free to check out the *Solution* below if you get stuck.

In [ ]:
brief_chatbot_system_message = "You always answer as briefly and concisely as possible."

curious_chatbot_system_message = """\
You are incredibly curious, and often respond with reflections and followup questions that lean the conversation in the direction of playfully \
understanding more about the subject matters of the conversation."""

increased_vocabulary_system_message = """\
You always respond using challenging and often under-utilized vocabulary words, even when your response could be made more simply."""

### Your Work Here

Update the following class definition so that the passed-in `system_message` is effectively utilized by chatbot instances.

In [ ]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))

        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

### Try Out a Chatbot With a Role

After successfully implementing `ChatbotWithRole`, try creating an instance of it with a system message of your choosing and interact with it.

### Solution

The solution is brief. Here we added an additional system message to the `chat_conversation_template` that uses the passed-in `system_message`.

In [ ]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('system', system_message),
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))

        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

Let's try it out with one of the system messages defined above.

In [ ]:
brief_chatbot = ChatbotWithRole(llm, system_message=brief_chatbot_system_message)
curious_chatbot = ChatbotWithRole(llm, system_message=curious_chatbot_system_message)
increased_vocabulary_chatbot = ChatbotWithRole(llm, system_message=increased_vocabulary_system_message)

In [ ]:
print(brief_chatbot.chat("What would you consider a good morning routine?"))

Wake up 15 minutes earlier than usual, stretch, drink water, meditate for 5 minutes, and exercise for 10 minutes.


In [ ]:
print(curious_chatbot.chat("What would you consider a good morning routine?"))

A good morning routine! It's like the foundation of a beautiful day, don't you think? I'm curious, what do you think makes a morning routine "good"? Is it about waking up early, or is it more about setting a positive tone for the day?

For me, I think a good morning routine is all about finding a balance between relaxation and productivity. I'd love to hear about your morning habits! Do you have a consistent routine, or do you like to mix things up?

Some people swear by meditation and yoga, while others prefer a good cup of coffee and a quick workout. And then there are those who just want to hit the snooze button and roll out of bed (no judgment here!).

What about you? What's your morning routine like?


In [ ]:
print(increased_vocabulary_chatbot.chat("What would you consider a good morning routine?"))

A salubrious morning routine, one that sets the tone for a diurnal existence replete with productivity and vitality, would likely involve a judicious combination of invigorating activities. I would recommend commencing the day with a refreshing ablution, followed by a brisk and invigorating exercise routine, such as a brisk jog or a series of dynamic yoga poses.

Subsequently, a nutritious and wholesome breakfast, replete with a balanced amalgam of complex carbohydrates, lean protein, and healthy fats, would be an excellent way to satiate the pangs of hunger and provide sustained energy throughout the morning. A stimulating cup of coffee or tea, imbued with the essence of aromatic spices, would also be a welcome addition to this morning ritual.

Furthermore, a brief period of mental preparation, involving a meditative or contemplative practice, such as mindfulness or journaling, would be an excellent way to clarify one's thoughts, focus the mind, and set intentions for the day ahead. T

---

## Gradio

_"Gradio is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!"_

If you find yourself building chatbots, especially for prototypes or even personal use chatbots, you might consider [Gradio](https://www.gradio.app/), which makes it easy to setup a pleasant chat interface, including in Jupyter environments.

Pass a `chatbot` instance (created with either the `Chatbot` class or `ChatbotWithRole` class) into the following `create_chatbot_interface` function and have a conversation. If you're interested, check out [chat_helpers/gradio_interface.py](chat_helpers/gradio_interface.py) for the source code.

In [ ]:
from chat_helpers.gradio_interface import create_chatbot_interface

In [ ]:
app = create_chatbot_interface(curious_chatbot)
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860


/usr/local/lib/python3.12/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://ea03ae2f4250a4b968.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


---

## Summary

In this notebook you learned how to leverage a new message type, a placeholder message, to create chatbots capable of retaining conversation history.

This was the final notebook in this section focused primarily on the explicit use of chat message types to benefit your LLM-based application code, and you learned a variety of techniques in addition to managing conversation history, like few-shot prompting, utilizing the system message, and performing chain-of-though prompting.

In this next section you will focus your attention on using a variety of prompt engineering techniques to enable your LLM-based applications to generated structured data, a powerful capability that unlocks the ability of your LLM-based applications to interact more immediately with downstream code, and opens incredible possibilities for using LLMs to tag and anaylyze large collections of textual data.